In [1]:
from logging import getLogger
from logging.config import dictConfig

from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from yaml import safe_load

from streamvigil import ADBenchDataset
from streamvigil.detectors import BasicAutoEncoder, BasicDetector

In [2]:
# Logger
with open("./logging.yml", encoding="utf-8") as file:
    config = safe_load(file)
dictConfig(config)
logger = getLogger(__name__)

In [3]:
# Anomaly Detector
auto_encoder = BasicAutoEncoder(
    encoder_dims=[500, 450, 400, 350, 300, 250],
    decoder_dims=[250, 300, 350, 400, 450, 500],
    batch_norm=True,
)
detector = BasicDetector(auto_encoder)

In [4]:
random_state = 42

# Load dataset
train_data = ADBenchDataset(
    "../data/9_census.npz",
    train=True,
    random_state=random_state,
)
test_data = ADBenchDataset(
    "../data/9_census.npz",
    train=False,
    random_state=random_state,
)

# DataLoader
train_loader = DataLoader(train_data, batch_size=512)
test_loader = DataLoader(test_data, batch_size=256)

In [5]:
# Training
epochs = 10
logger.info("Start training the model...")
for epoch in range(epochs):
    logger.info(f"Epoch: {epoch+1}")
    for batch, (X, _) in enumerate(train_loader):
        loss = detector.train(X)

        if batch % 100 == 0:
            logger.info(f"Loss: {loss.item():>7f}")
logger.info("Completed training the model!")

2024-05-22 17:31:03,106 [INFO] __main__: Start training the model...
2024-05-22 17:31:03,107 [INFO] __main__: Epoch: 1
2024-05-22 17:31:03,567 [INFO] __main__: Loss: 0.228254
2024-05-22 17:31:03,995 [INFO] __main__: Loss: 0.014057
2024-05-22 17:31:04,423 [INFO] __main__: Loss: 0.011099
2024-05-22 17:31:04,832 [INFO] __main__: Loss: 0.009813
2024-05-22 17:31:05,251 [INFO] __main__: Loss: 0.009438
2024-05-22 17:31:05,339 [INFO] __main__: Epoch: 2
2024-05-22 17:31:05,344 [INFO] __main__: Loss: 0.009452
2024-05-22 17:31:05,756 [INFO] __main__: Loss: 0.008942
2024-05-22 17:31:06,179 [INFO] __main__: Loss: 0.007667
2024-05-22 17:31:06,646 [INFO] __main__: Loss: 0.007233
2024-05-22 17:31:07,055 [INFO] __main__: Loss: 0.007206
2024-05-22 17:31:07,091 [INFO] __main__: Epoch: 3
2024-05-22 17:31:07,096 [INFO] __main__: Loss: 0.007392
2024-05-22 17:31:07,511 [INFO] __main__: Loss: 0.007246
2024-05-22 17:31:07,920 [INFO] __main__: Loss: 0.006331
2024-05-22 17:31:08,397 [INFO] __main__: Loss: 0.0060

In [6]:
# Evaluation
metrics = BinaryAUROC()
logger.info("Start evaluating the model...")

for X, y in test_loader:
    scores = detector.predict(X)
    metrics.update(scores, y)

# Compute AUROC score   
logger.info(f"AUROC Score: {metrics.compute()}")

logger.info("Completed the evaluation of the model!")

2024-05-22 17:31:21,310 [INFO] __main__: Start evaluating the model...
2024-05-22 17:31:22,000 [INFO] __main__: AUROC Score: 0.6405672835707877
2024-05-22 17:31:22,001 [INFO] __main__: Completed the evaluation of the model!
